<a href="https://colab.research.google.com/github/akshayugalmogale/3d-Printing-Regression-Model/blob/main/Predict_Elongation_of_3D_Print_Material.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [86]:
printer = pd.read_csv('/content/3DPrinting_data.csv')

In [87]:
printer['infill_pattern'].replace(['grid','honeycomb'], [0,1], inplace = True)
printer['material'].replace(['abs','pla'], [0,1], inplace = True)

In [88]:
#importing statsmodels library
import statsmodels.api as sm
# let's define a function for the multiple regression

def linear_Regression(x,y):
    
    x = sm.add_constant(x)
    
    #defining the model, fitting the model and printing the results
    multiple_model = sm.OLS(y,x).fit()
    print(multiple_model.summary())

# Model 3 : Predicitng Elongation based on the features

In [89]:
printer.head()

,layer_height,wall_thickness,infill_density,infill_pattern,nozzle_temperature,bed_temperature,print_speed,material,fan_speed,roughness,tension_strength,elongation
0,0.02,8,90,0,220,60,40,0,0,25,18,1.2
1,0.02,7,90,1,225,65,40,0,25,32,16,1.4
2,0.02,1,80,0,230,70,40,0,50,40,8,0.8
3,0.02,4,70,1,240,75,40,0,75,68,10,0.5
4,0.02,6,90,0,250,80,40,0,100,92,5,0.7


In [90]:
X = printer.drop(['roughness','tension_strenght','elongation'], axis = 1)
y = printer['tension_strenght']
X.head()

,layer_height,wall_thickness,infill_density,infill_pattern,nozzle_temperature,bed_temperature,print_speed,material,fan_speed
0,0.02,8,90,0,220,60,40,0,0
1,0.02,7,90,1,225,65,40,0,25
2,0.02,1,80,0,230,70,40,0,50
3,0.02,4,70,1,240,75,40,0,75
4,0.02,6,90,0,250,80,40,0,100


In [91]:
linear_Regression(X,y)

                            OLS Regression Results                            
Dep. Variable:       tension_strenght   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.609
Method:                 Least Squares   F-statistic:                     10.55
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           6.91e-08
Time:                        16:17:11   Log-Likelihood:                -151.94
No. Observations:                  50   AIC:                             321.9
Df Residuals:                      41   BIC:                             339.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0663      0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [92]:
X = X.drop(['infill_pattern','print_speed'], axis = 1)
X.head()

,layer_height,wall_thickness,infill_density,nozzle_temperature,bed_temperature,material,fan_speed
0,0.02,8,90,220,60,0,0
1,0.02,7,90,225,65,0,25
2,0.02,1,80,230,70,0,50
3,0.02,4,70,240,75,0,75
4,0.02,6,90,250,80,0,100


In [93]:
linear_Regression(X,y)

                            OLS Regression Results                            
Dep. Variable:       tension_strenght   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.620
Method:                 Least Squares   F-statistic:                     14.33
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           6.78e-09
Time:                        16:17:11   Log-Likelihood:                -152.42
No. Observations:                  50   AIC:                             318.8
Df Residuals:                      43   BIC:                             332.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0646      0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [94]:
#get the interaction terms by multiplying values

inter_mn = X['material']*X['nozzle_temperature']
inter_bn = X['bed_temperature']*X['nozzle_temperature']
inter_fn = X['fan_speed']*X['nozzle_temperature']
inter_fb = X['fan_speed']*X['bed_temperature']

In [95]:
#adding these interaction terms to dataset using .concat() function of pandas
#we will call this dataset as interaction

interaction = pd.concat([X,inter_mn,inter_bn,inter_fn,inter_fb], axis = 1)

#chenge column names of this interaction terms
interaction = interaction.rename(columns = {0:'interct_mn', 1:'interact_bn', 2:'interact_fn',
                             3:'interact_fb'})

interaction.head(10)

,layer_height,wall_thickness,infill_density,nozzle_temperature,bed_temperature,material,fan_speed,interct_mn,interact_bn,interact_fn,interact_fb
0,0.02,8,90,220,60,0,0,0,13200,0,0
1,0.02,7,90,225,65,0,25,0,14625,5625,1625
2,0.02,1,80,230,70,0,50,0,16100,11500,3500
3,0.02,4,70,240,75,0,75,0,18000,18000,5625
4,0.02,6,90,250,80,0,100,0,20000,25000,8000
5,0.02,10,40,200,60,1,0,200,12000,0,0
6,0.02,5,10,205,65,1,25,205,13325,5125,1625
7,0.02,10,10,210,70,1,50,210,14700,10500,3500
8,0.02,9,70,215,75,1,75,215,16125,16125,5625
9,0.02,8,40,220,80,1,100,220,17600,22000,8000


In [96]:
interaction = pd.concat([X,inter_mn,inter_bn,inter_fn,inter_fb], axis = 1)

interaction = interaction.rename(columns = {0:'interct_mn', 1:'interact_bn', 2:'interact_fn',
                             3:'interact_fb'})

interaction.head()

,layer_height,wall_thickness,infill_density,nozzle_temperature,bed_temperature,material,fan_speed,interct_mn,interact_bn,interact_fn,interact_fb
0,0.02,8,90,220,60,0,0,0,13200,0,0
1,0.02,7,90,225,65,0,25,0,14625,5625,1625
2,0.02,1,80,230,70,0,50,0,16100,11500,3500
3,0.02,4,70,240,75,0,75,0,18000,18000,5625
4,0.02,6,90,250,80,0,100,0,20000,25000,8000


In [97]:
linear_Regression(interaction,y)

                            OLS Regression Results                            
Dep. Variable:       tension_strenght   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.654
Method:                 Least Squares   F-statistic:                     11.30
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           1.54e-08
Time:                        16:17:11   Log-Likelihood:                -148.26
No. Observations:                  50   AIC:                             316.5
Df Residuals:                      40   BIC:                             335.6
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0781      0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


## Elonagtion and Tension_strength are strongly correlated to one another and both don't behave linearly.